# Application pour récupérer les lampadaires à partir d'un bbox depuis Mapillary: 
https://www.mapillary.com/developer/api-documentation/points?locale=fr_FR

In [8]:
import mercantile
import requests
from vt2geojson.tools import vt_bytes_to_geojson
from concurrent.futures import ThreadPoolExecutor
from shapely.geometry import box, shape
import geopandas as gpd
import json
import functools

# Choix de type de tuiles vectorielles
def choose_data_type(data_type):
    """
    Choisie le type des entités à récupérer
    
    Cette fonction prend en entrée le type de données tuiles des entités à récupérer ("tile_points" pour les points ou "tile_traffic_signs" pour
    les paneaux de signalisation) et retourne le type des entités à récupérer.
    
    Paramètres :
    - data_type (str) : le type de données tuiles des entités à récupérer.

    Retour :
    - 'mly_map_feature_point' (str) : la clé mapillary décrivant les entités "points".
    - 'mly_map_feature_traffic_sign' (str) : la clé mapillary décrivant les entités "traffic_signs".
    """
    if (data_type == "tile_points"):
        return("mly_map_feature_point")
    elif (data_type == "tile_traffic_signs"):
        return("mly_map_feature_traffic_sign")

# Charger le fichier tabulaire contenant la clé Token d'accès Mapillary
with open("../notebooks/mapillary_token.txt", encoding="utf-8") as f:
    access_token = f.read()

# Les coordonnées du Bounding box
(W, S, E, N) = (5.389065, 43.295426, 5.411281, 43.317027)

# Fonction pour filtrer les entités qui intersectent le polygone de la bbox
def filter_features(features, polygon):

    """
    Récupérer les entités qui intersectent un polygone.
    
    Cette fonction prend en entrée les dictionnaires des données des entités et un polygone d'intersection et retourne les entités qui intersectent
    le polygone.
    
    Paramètres :
    - feature (dict) : dictionnaire des données des entités.
    - polygon (shapely.geometry.Polygon) : polygone d'intersection.

    Retour :
    - filtered_features (list) : liste des entités qui intersectent le polygone.
    """
    filtered_features = []
    for feature in features:
        geom = shape(feature['geometry']) #convertir le GeoJSON de géométrie en objet vecteur

        if geom.intersects(polygon):
            filtered_features.append(feature)
    return filtered_features

# Fonction pour récupérer les données d'une tuile vectorielle
def fetch_tile(tile, data_type):
    """
    Chercher et récupérer les données des entités à partir d'une tuile vectorielle.
    
    Cette fonction prend en entrée une tuile vectorielle ainsi que le type de données tuiles à récupérer et retourne le dictionnaire des données
    des entités contenues dans cette tuile.
    
    Paramètres :
    - tile (mercantile.Tile) : tuile vectorielle.
    - data_type (str) : le type de données tuiles des entités à récupérer.

    Retour :
    - data['features'] (dict) : dictionnaire des données des entités contenues dans la tuile.
    """
    tile_url = f'https://tiles.mapillary.com/maps/vtp/{choose_data_type(data_type)}/2/{tile.z}/{tile.x}/{tile.y}?access_token={access_token}'
    response = requests.get(tile_url)
    data = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z)

    #choisir les type selon les valeurs des entités
    filtered_data = [feature for feature in data['features'] if feature['properties']['value'] in filter_values]
    return (filtered_data)

# Récupérer les données (selon choix : points ou panneaux de signalisation) depuis l'API de Mapillary
def get_mapillary_data(data_type, access_token, W, S, E, N,output_folder):

    """
    Récupérer les données de Mapillary à partir d'une bbox.
    
    Cette fonction prend en entrée le type de données tuiles des entités à récupérer ("tile_points" pour les points ou "tile_traffic_signs" pour
    les paneaux de signalisation), la clé Token d'accès Mapillary, ainsi que les coordonnées du bbox à partir duquel ces données sont extraites
    et retourne les données Mapillary catégorie "Points" incluses dans cette bbox.
    
    Paramètres :
    - data_type (str) : le type de données tuiles des entités à récupérer.
    - access_token (str) : la clé Token d'accès Mapillary.
    - W (float) : Longitude max de la bbox.
    - S (float) : Latitude min de la bbox.
    - E (float) : Longitude min de la bbox.
    - N (float) : Latitude max de la bbox.

    Retour :
    - gdf_mapillary (geopandas.GeoDataFrame) : Le GeoDataFrame des données Mapillary extraites.
    - gdf_bbox (geopandas.GeoDataFrame) : Le GeoDataFrame de la bbox.
    """
    
    # Définir un GeoJSON vide en tant que sortie
    output = {"type": "FeatureCollection", "features": []}

    # Récupérer la liste des tuiles intersectant notre emprise
    tiles = list(mercantile.tiles(W, S, E, N, 14))

    # Créer un polygone à partir de l'emprise
    bbox_polygon = box(W, S, E, N)
    gdf_bbox = gpd.GeoDataFrame({'geometry': [bbox_polygon]}, crs='EPSG:4326')

    # Utilisation de functools.partial pour fixer la valeur de l'argument 'data_type' dans la fonction fetch_tile
    partial_fetch_tile = functools.partial(fetch_tile, data_type=data_type)

    # Exécuter les requêtes en parallèle vu le nombre important des tuiles
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(partial_fetch_tile, tiles))

    # Filtrer les resultats des entités contenues dans l
    for feature in results:
        filtered_features = filter_features(feature, bbox_polygon)
        output['features'].extend(filtered_features)

    # Export du fichier résultat sous fromat geojson
    with open(output_folder, "w") as fx:
        json.dump(output, fx)
        gdf_mapillary = gpd.GeoDataFrame.from_features(output)
    return(gdf_mapillary,gdf_bbox)

# Visualisation :
from ipyleaflet import Map, GeoData, ZoomControl

def render_map(data, pol):

    """
    Visualsation des données OSM extraites à partir d'un polygone quelconque (bbox ou buffer ou contour de commune/quartier...).
    
    Cette procedure prend en entrée les données OSM ainsi que le polygone à partir duquel ces données sont extraites et retourne
    les données OSM catégorie "transport" incluses dans cette zone.
    
    Paramètres :
    - data (geopandas.GeoDataFrame) : Le GeoDataFrame des données OSM extraites.
    - pol (geopandas.GeoDataFrame) : Le GeoDataFrame du polygone d'extraction. 
    """    
    
    m = Map(center=(pol.bounds['miny'].values[0],pol.bounds['minx'].values[0]))

    # Visualisation des données extraites
    geo_data1 = GeoData(geo_dataframe = data,
        style={'color': 'black', 'radius':3, 'fillColor': '#cc3333', 'opacity':1, 'weight':1, 'dashArray':'2', 'fillOpacity':1},
        hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
        point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
        name = 'Release')

    # Visualisation du polygone d'extraction
    geo_data2 = GeoData(geo_dataframe = pol,
        style={'color': 'blue', 'radius':2, 'fillColor': '#004dff', 'opacity':1, 'weight':2, 'dashArray':'1', 'fillOpacity':0})
    
    m.add(geo_data2)
    m.add(geo_data1)

    m.fit_bounds([[pol.bounds['miny'].values[0], pol.bounds['minx'].values[0]], # Ajuster le zoom et centrer la carte pour afficher les limites du polygone
              [pol.bounds['maxy'].values[0], pol.bounds['maxx'].values[0]]])
    
    display(m)

data_type = "tile_points"
filter_values = ["object--street-light"]
Name = "Eclairage_Mly_bbox"
output_folder = "../open_data/processed/"+Name+".geojson"
Eclairage_mapillary,bbox = get_mapillary_data(data_type, access_token, W, S, E, N, output_folder)
print("end")
render_map(Eclairage_mapillary, bbox)

end


Map(center=[43.295426, 5.389065], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [9]:
Eclairage_mapillary

,geometry,first_seen_at,id,last_seen_at,value
0,POINT (5.40250 43.31351),1510242627000,522681202093720,1655810162000,object--street-light
1,POINT (5.40332 43.31084),1655810304000,159832899917300,1655810304000,object--street-light
2,POINT (5.40293 43.30966),1655810334000,523700902881023,1655810334000,object--street-light
3,POINT (5.40380 43.31597),1510242461000,925167651663371,1510242461000,object--street-light
4,POINT (5.40049 43.31095),1655809169000,345480270920024,1655809169000,object--street-light
...,...,...,...,...,...
1499,POINT (5.40711 43.30374),1510245343000,1607210409475759,1510245343000,object--street-light
1500,POINT (5.40615 43.29626),1655970839000,705162300783560,1655970839000,object--street-light
1501,POINT (5.40621 43.29612),1655970837000,705162290783561,1655970837000,object--street-light
1502,POINT (5.40938 43.30422),1510245221000,2908693776068497,1510245221000,object--street-light
